In [40]:
%load_ext autoreload
%autoreload 2

import numpy as np
import numba as nb
import scipy as sp
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
from pathlib import Path
import vrExperiment as vre
import vrFunctions as vrf
import basicFunctions as bf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Steps to code: 
# 1. consider replacing some of the functions in "multiSessionAnalysis"...
# 2. add red cell processing functions into the pipeline

# immediate goals: 
# 1. write functions for retrieving useful variables from oneData formats (e.g. speed, occupancy maps, etc.)
# 2. do I save these? how long will it take to load? 

# BIG GOAL:
# 1. Database time! Do this then rerun all your sessions
# 2. Create smart loading system where each vrexp object loads one data into a floating dictionary, and every time something is needed it checks if it's loaded it and then loads it only when necessary
#    For this to work, I'll have to exclusively use functions that are called on each vrexp object... it's a little tricky but I think it can work

# -- before databasing, I want to test transformed data performance on a few sessions, i.e. I want to see how quickly I can regenerate key behavioral and neural variables from the onedata 

In [41]:
# vrExperiment registration and preprocessing 
mouseName = 'ATL022'
dateString = '2023-03-24'
session = '701'
vrReg = vre.vrExperimentRegistration(mouseName, dateString, session, oasis=False, imaging=True)
vrReg.doPreprocessing()
vrReg.saveParams()
print(vrReg.sessionPath())

In session: ATL022/2023-03-24/701, vrFile['rigInfo'] does not exist. Assuming default settings for B2!
Self.value['numTrials'] set by trialInfo['trialIdx']>0, but this might not be right. There might be smarter ways to determine which trials are 'good' trials...
Facecam preprocessing has not been coded yet!
C:\Users\andrew\Documents\localData\ATL022\2023-03-24\701


In [162]:
# Post-processing summary variables
# 1. convolve spks ==========================Note: I don't think I'm going to keep doing this. It barely changes the spike vector and we're already using GCaMP data anyway...
# 2. occupancy map, speed map, lick map ================ finished
# 3. spkmap for position (with spatial smoothing) =================== finished (and I wrote a toeplitz convolution that's fast)
# 4. reliability measurements and spatial information measurements 
# 5. ROICaT alignment tools 

# create all these variables (make sure you did it correctly), and time how long it takes to load them
# then save them all and time it takes how long to load them and how much data to save them

In [42]:
mouseName = 'ATL022'
dateString = '2023-03-24'
session = '701'
vrexp = vre.vrExperiment(mouseName, dateString, session)

In [154]:
%%time
# Retrieve smoothed and downsampled occupancymap and speedmap
omap, smap, distvec = vrf.getBehaviorMaps(vrexp,speedThreshold=5) # produce smoothed occupancy and speed maps, along with the distance bins used to create them
frameTrialIdx, framePosition, frameSpeed = vrexp.getFrameBehavior()
spkmap, count = vrf.getSpikeMap(vrexp, frameTrialIdx, framePosition, frameSpeed, distvec, omap, standardizeSpks=False, doSmoothing=None) # produce the spkmap 
idxFullTrials = np.all(~np.isnan(omap),axis=1)

Lick map should be included in 'getBehaviorMaps' eventually!
CPU times: total: 3.8 s
Wall time: 226 ms


In [206]:
%time relmse, relcor = vrf.measureReliability(spkmap[:,idxFullTrials], numcv=3)

CPU times: total: 203 ms
Wall time: 210 ms


In [195]:
%time spInfo = vrf.measureSpatialInformation(omap[idxFullTrials], spkmap[:,idxFullTrials])

CPU times: total: 62.5 ms
Wall time: 73.8 ms
